In [1]:
from IPython.display import HTML

HTML("""
<button id="code-show-switch-btn">Hide Script</button>

<script>
var code_show = true;

function switch_display_setting() {
    var switch_btn = $("#code-show-switch-btn");
    if (code_show) {
        $("div.input").hide();
        code_show = false;
        switch_btn.text("Display Script");
    }else {
        $("div.input").show();
        code_show = true;
        switch_btn.text("Hide Script");
    }
}

$("#code-show-switch-btn").click(switch_display_setting);
</script>
""")

In [ ]:
# For mac-OS environment 
import stata_setup
stata_setup.config("/Applications/stata", "mp")

In [15]:
# For windows environment 
import numpy as np
import matplotlib.pyplot as plt
import markdown
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.options.display.precision = 4
import stata_setup
stata_setup.config("C://Program Files//Stata17", "be")
md = markdown.Markdown(extensions=['pymdownx.tabbed'])

# Setup

In [16]:
%%stata
qui{
use ../data/suicide_analysis_data_static,clear
xtset citycode covid  

/* Local macro */
local absvars="citycode covid"
local clsvars="citycode"
local absvars_ind "i.citycode i.covid"    
local gmm2="gmm2s"
local log_="log_"
sum suicide
local suicide_t_m=r(mean)
sum suicide_m
local suicide_m_m=r(mean)
sum suicide_f
local suicide_f_m=r(mean)
sum `log_'active_opening_ratio
local log_active_opening_ratio_m=r(mean)
sum `log_'unemployment_rate
local log_unemployment_rate_m=r(mean)
sum `log_'patient
local log_patient_m=r(mean)
sum log_active_opening_weighted
local log_active_opening_weighted_m=r(mean)
sum log_bankruptcy_debt
local log_bankruptcy_debt_m=r(mean)

* For Tables
local txt_u20="Under 20"
local txt_20_40="20-39"
local txt_40_60="40-59"
local txt_o60="60 or over"

/* Transformation of log(1+var) */
#delimit ;    

local paracomp_f_mobility="(mobility_index: _b[D.mobility_index]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_f_cv="(cv_residents: _b[D.cv_residents]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_f_1st="(commuter_ratio_m: _b[commuter_ratio_m]) ";
local paracomp_f_cv_1st="(commuter_ratio_m: _b[commuter_ratio_m]) ";
local paracomp_f_cv_reduced="(commuter_ratio_m: _b[commuter_ratio_m]*(1+`suicide_f_m')/(`suicide_f_m'))  ";
local paracomp_f_reduced="(commuter_ratio_m: _b[commuter_ratio_m]*(1+`suicide_f_m')/(`suicide_f_m'))  ";

local paracomp_m_mobility="(mobility_index: _b[D.mobility_index]*(1+`suicide_m_m')/(`suicide_m_m')) ";
local paracomp_m_cv="(cv_residents: _b[D.cv_residents]*(1+`suicide_m_m')/(`suicide_m_m')) ";
local paracomp_m_cv_1st="(commuter_ratio_f: _b[commuter_ratio_f]) ";
local paracomp_m_1st="(commuter_ratio_f: _b[commuter_ratio_f]) ";
local paracomp_m_cv_reduced="(commuter_ratio_f: _b[commuter_ratio_f]*(1+`suicide_m_m')/(`suicide_m_m')) ";
local paracomp_m_reduced="(commuter_ratio_f: _b[commuter_ratio_f]*(1+`suicide_m_m')/(`suicide_m_m')) ";
};



. qui{

. 


# Summary Statistics

In [17]:
%%stata

qui{
label variable suicide "Suicide"
label variable suicide_f "Female suicide"
label variable suicide_m "Male suicide"
label variable active_opening_ratio "Active opening ratio"
label variable bankruptcy_debt "Bankruptcy debt"
label variable patient "Number of patinets (per month)"

    
eststo before: quietly estpost summarize ///
    suicide suicide_f suicide_m mobility_index cv_residents cv_nonresidents active_opening_ratio patient commuter_ratio commuter_ratio_f commuter_ratio_m if covid == 0
eststo after: quietly estpost summarize ///
    suicide suicide_f suicide_m mobility_index cv_residents cv_nonresidents active_opening_ratio patient commuter_ratio commuter_ratio_f commuter_ratio_m if covid == 1
eststo total: quietly estpost summarize ///
    suicide suicide_f suicide_m mobility_index cv_residents cv_nonresidents active_opening_ratio patient commuter_ratio commuter_ratio_f commuter_ratio_m
}

esttab total before after, ///
cells("mean(pattern(1 1 1) fmt(%4.3f)) sd(pattern(1 1 1) fmt(%4.3f)) min(pattern(1 0 0) fmt(%4.3f)) max(pattern(1 0 0) fmt(%4.3f))") ///
nomtitles mgroups("Total" "Before" "After", pattern(1 1 1)) label nonumber rename(cv_residents "Coefficient of variation (CV)") ///
varwidth(20) modelwidth(6)
    
esttab total before after using "../figuretable/appendix/summary_statistic.tex", replace rename(cv_residents "Coefficient of variation (CV)") ///
cells("mean(pattern(1 1 1) fmt(%4.3f)) sd(pattern(1 1 1) fmt(%4.3f)) min(pattern(1 0 0) fmt(%4.3f)) max(pattern(1 0 0) fmt(%4.3f))") ///
nomtitles noobs mgroups("Total" "Before" "After", pattern(1 1 1) prefix(\multicolumn{@span}{c}{) suffix(}) span erepeat(\cmidrule(lr){@span})) label nonumber ///
/*Caveats: Hard Cording Observations!*/ ///
prefoot("\hline \multicolumn{1}{c}{Observations} & \multicolumn{4}{c}{3782} & \multicolumn{2}{c}{1891} & \multicolumn{2}{c}{1891}\\") /// 
booktabs alignment(D{.}{.}{-1})


. 
. qui{

. 
. esttab total before after, ///
> cells("mean(pattern(1 1 1) fmt(%4.3f)) sd(pattern(1 1 1) fmt(%4.3f)) min(patt
> ern(1 0 0) fmt(%4.3f)) max(pattern(1 0 0) fmt(%4.3f))") ///
> nomtitles mgroups("Total" "Before" "After", pattern(1 1 1)) label nonumber re
> name(cv_residents "Coefficient of variation (CV)") ///
> varwidth(20) modelwidth(6)

----------------------------------------------------------------------------
                      Total                      Before         After       
                       mean     sd    min    max   mean     sd   mean     sd
----------------------------------------------------------------------------
Suicide              10.815 15.642  0.000 123.000 10.364 14.818 11.266 16.417
Female suicide        3.516  5.582  0.000 48.000  3.147  4.943  3.885  6.134
Male suicide          7.299 10.447  0.000 87.000  7.217 10.212  7.381 10.679
Outflow ratio         0.316  0.140  0.000  0.870  0.342  0.146  0.291  0.129
Coefficient of var~) -0.00

# Female All Age: Outflow ratio

## Female: Full Result with covariates

In [18]:
%%stata
qui{
foreach x in u20 20_40 40_60 o60{
sum suicide_f_`x'
local suicide_f_m=r(mean)
    
* Define label temporary    
label variable mobility_index "Mobility index"
label variable cv_residents "Mobility index"

#delimit ;
local paracomp_f_mobility_full="(mobility_index: _b[D.mobility_index]*(1+`suicide_f_m')/(`suicide_f_m'))
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))*((1+`suicide_f_m')/(`suicide_f_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))*((1+`suicide_f_m')/(`suicide_f_m'))) ";
local paracomp_f_cv_full="(cv_residents: _b[D.cv_residents]*(1+`suicide_f_m')/(`suicide_f_m')) 
    (cv_nonresidents: _b[D.cv_nonresidents]*(1+`suicide_f_m')/(`suicide_f_m'))
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))*((1+`suicide_f_m')/(`suicide_f_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))*((1+`suicide_f_m')/(`suicide_f_m'))) ";
local paracomp_f_1st_full="(commuter_ratio_m: _b[commuter_ratio_m]) 
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))) ";    
local paracomp_f_cv_1st_full="(commuter_ratio_m: _b[commuter_ratio_m]) 
    (cv_nonresidents: _b[D.cv_nonresidents]*(1+`suicide_f_m')/(`suicide_f_m'))
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))) ";    
local paracomp_f_cv_reduced_full="(commuter_ratio_m: _b[commuter_ratio_m]*(1+`suicide_f_m')/(`suicide_f_m')) 
    (cv_nonresidents: _b[D.cv_nonresidents]*(1+`suicide_f_m')/(`suicide_f_m'))
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))*((1+`suicide_f_m')/(`suicide_f_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))*((1+`suicide_f_m')/(`suicide_f_m'))) ";
local paracomp_f_reduced_full="(commuter_ratio_m: _b[commuter_ratio_m]*(1+`suicide_f_m')/(`suicide_f_m')) 
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))*((1+`suicide_f_m')/(`suicide_f_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))*((1+`suicide_f_m')/(`suicide_f_m'))) ";

#delimit cr
/* Outflow ratio */
* FE
reghdfe D.suicide_f_`x'_log D.mobility_index D.`log_'active_opening_ratio D.`log_'patient, cluster(`clsvars')
nlcom `paracomp_f_mobility_full', post
est store suicide_f_`x'_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_f_`x'_log (D.mobility_index=commuter_ratio_m  ) D.`log_'active_opening_ratio D.`log_'patient, gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_f_`x'
    
nlcom `paracomp_f_mobility_full', post
est store suicide_f_`x'_2nd
    
** Save 1st stage result
est restore F_1st_D_mobility_index
nlcom `paracomp_f_1st_full', post
est store suicide_f_`x'_1st

** Add 1st stage statistics
est restore tmp_suicide_f_`x'
estadd scalar F = `e(widstat)' : suicide_f_`x'_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_f_`x'_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_`x'_1st

** Reduced form
reghdfe D.suicide_f_`x'_log commuter_ratio_m D.`log_'active_opening_ratio D.`log_'patient, cluster(`clsvars')
nlcom `paracomp_f_reduced_full', post
est store suicide_f_`x'_reduced

/* CV */
* FE
reghdfe D.suicide_f_`x'_log D.cv_residents D.cv_nonresidents D.`log_'active_opening_ratio D.`log_'patient, cluster(`clsvars')
nlcom `paracomp_f_cv_full', post
est store suicide_f_`x'_cv_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_f_`x'_log (D.cv_residents=commuter_ratio_m  ) D.cv_nonresidents D.`log_'active_opening_ratio D.`log_'patient, gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_f_`x'_cv

nlcom `paracomp_f_cv_full', post
est store suicide_f_`x'_cv_2nd
    
** Save 1st stage result
est restore F_1st_D_cv_residents
nlcom `paracomp_f_cv_1st_full', post
est store suicide_f_`x'_cv_1st

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_cv
estadd scalar F = `e(widstat)' : suicide_f_`x'_cv_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_f_`x'_cv_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_`x'_cv_1st    
      

** Reduced form
reghdfe D.suicide_f_log commuter_ratio_m D.cv_nonresidents D.`log_'active_opening_ratio D.`log_'patient, cluster(`clsvars')
nlcom `paracomp_f_cv_reduced_full', post
est store suicide_f_`x'_cv_reduced      
                                                                                               
#delimit ;
noi: di "******* Female `x' *******";                                                                                               
noi: esttab suicide_f_`x'_FE suicide_f_`x'_1st suicide_f_`x'_2nd suicide_f_`x'_reduced suicide_f_`x'_cv_FE suicide_f_`x'_cv_1st suicide_f_`x'_cv_2nd suicide_f_`x'_cv_reduced, 
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) label rename(cv_residents mobility_index)
keep(mobility_index cv_nonresidents commuter_ratio_m `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_m)                                                                                              
mgroups("Outflow Ratio" "CV", pattern(1 0 0 0 1 0 0 0)) mlabels("OLS" "1st Stage" "IV" "Reduced form" "OLS" "1st Stage" "IV" "Reduced form") varwidth(4) modelwidth(4)
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation"));                                                                                                                                                                                     
 
* Redefine label                                                                                 
label variable mobility_index "Outflow ratio";
label variable cv_residents "CV";                                                                                         
                                                                                               
};                                                                                                                                                                                     
};


. qui{
******* Female u20 *******

--------------------------------------------------------------------
     Out~o                              CV                           
      (1)     (2)     (3)     (4)     (5)     (6)     (7)     (8)   
      OLS    1st~e      IV    Red~m     OLS    1st~e      IV    Red~m   
--------------------------------------------------------------------
Mob~x -2.436**          -5.254**          -0.319**          -0.616**         
     (1.089)            (2.529)            (0.135)            (0.292)          
>  

CV ~s                                 0.031    0.158    0.056    0.055   
                                     (0.055)    (0.165)    (0.058)    (0.172)  
>  

Log~) -0.033    0.004*** 0.002    -0.020    -0.029    0.036*** 0.003    0.033  
>  
     (0.089)    (0.001)    (0.093)    (0.091)    (0.089)    (0.012)    (0.093) 
>    (0.163)   

Log~) 0.028    -0.003*** 0.008    0.027    0.027    -0.029*** 0.010    0.033   
     (0.030)    (0.001)    (0.0

In [19]:
%%stata
* Export stacked tables
qui{ 

* Define label temporary    
label variable mobility_index "Mobility index"
label variable cv_residents "Mobility index"
    
* female u20
esttab suicide_f_u20_FE suicide_f_u20_1st suicide_f_u20_2nd suicide_f_u20_reduced suicide_f_u20_cv_FE suicide_f_u20_cv_1st suicide_f_u20_cv_2nd suicide_f_u20_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_female.tex", ///
prehead("\begin{tabular}{l*{8}{c}} \toprule") ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(A)Under 20}}} \\\\[-1ex]") ///    
fragment ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index cv_nonresidents commuter_ratio_m `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_m) ///                                                                                                                                                                                                                                                                                 
mgroups("Outflow Ratio" "CV", pattern(1 0 0 0 1 0 0 0) prefix(\multicolumn{@span}{c}{) suffix(}) span erepeat(\cmidrule(lr){@span})) ///
mlabels("OLS" "1st Stage" "IV" "Reduced form" "OLS" "1st Stage" "IV" "Reduced form") ///                            
nomtitles noobs label rename(cv_residents mobility_index) ///
booktabs alignment(D{.}{.}{-1}) ///                                                                                  
replace
                                                                                 
* female 20-40
esttab suicide_f_20_40_FE suicide_f_20_40_1st suicide_f_20_40_2nd suicide_f_20_40_reduced suicide_f_20_40_cv_FE suicide_f_20_40_cv_1st suicide_f_20_40_cv_2nd suicide_f_20_40_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_female.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(B)20-39}}} \\\\[-1ex]") ///
fragment ///   
append ///                                                                                    
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index cv_nonresidents commuter_ratio_m `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_m) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers noobs label nolines rename(cv_residents mobility_index) ///   
booktabs alignment(D{.}{.}{-1}) ///                                                                                  
                                                                                 
* female 40-60
esttab suicide_f_40_60_FE suicide_f_40_60_1st suicide_f_40_60_2nd suicide_f_40_60_reduced suicide_f_40_60_cv_FE suicide_f_40_60_cv_1st suicide_f_40_60_cv_2nd suicide_f_40_60_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_female.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(C)40-59}}} \\\\[-1ex]") ///
fragment ///   
append ///                                                                                    
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index cv_nonresidents commuter_ratio_m `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_m) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers noobs label nolines rename(cv_residents mobility_index) /// 
booktabs alignment(D{.}{.}{-1}) ///                                                                                 
                                                                                 
                                                                                     
* female o60                                                                                  
esttab suicide_f_o60_FE suicide_f_o60_1st suicide_f_o60_2nd suicide_f_o60_reduced suicide_f_o60_cv_FE suicide_f_o60_cv_1st suicide_f_o60_cv_2nd suicide_f_o60_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_female.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(D)Over 60}}} \\\\[-1ex]") ///
fragment ///
append ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///           
keep(mobility_index cv_nonresidents commuter_ratio_m `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_m) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers label rename(cv_residents mobility_index) ///
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation")) ///                                                                                    
prefoot("\hline") ///                                                                                   
postfoot("\bottomrule \multicolumn{9}{l}{\footnotesize Standard errors in parentheses}\\ \multicolumn{8}{l}{\footnotesize \sym{*} \(p<0.10\), \sym{**} \(p<0.05\), \sym{***} \(p<0.01\)} \end{tabular}") ///
booktabs alignment(D{.}{.}{-1})

        
* Redefine label                                                                                 
label variable mobility_index "Outflow ratio"
label variable cv_residents "CV"
                                                                                 
}



. * Export stacked tables
. qui{ 

. 


## Female: Full Result without covariates

In [20]:
%%stata
qui{
foreach x in u20 20_40 40_60 o60{
sum suicide_f_`x'
local suicide_f_m=r(mean)
    
* Define label temporary    
label variable mobility_index "Mobility index"
label variable cv_residents "Mobility index"

#delimit ;
local paracomp_f_mobility="(mobility_index: _b[D.mobility_index]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_f_cv="(cv_residents: _b[D.cv_residents]*(1+`suicide_f_m')/(`suicide_f_m'))";
local paracomp_f_1st="(commuter_ratio_m: _b[commuter_ratio_m]) ";    
local paracomp_f_cv_1st="(commuter_ratio_m: _b[commuter_ratio_m]) ";    
local paracomp_f_cv_reduced="(commuter_ratio_m: _b[commuter_ratio_m]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_f_reduced="(commuter_ratio_m: _b[commuter_ratio_m]*(1+`suicide_f_m')/(`suicide_f_m')) ";

#delimit cr
/* Outflow ratio */
* FE
reghdfe D.suicide_f_`x'_log D.mobility_index, cluster(`clsvars')
nlcom `paracomp_f_mobility', post
est store suicide_f_`x'_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_f_`x'_log (D.mobility_index=commuter_ratio_m), gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_f_`x'
    
nlcom `paracomp_f_mobility', post
est store suicide_f_`x'_2nd
    
** Save 1st stage result
est restore F_1st_D_mobility_index
nlcom `paracomp_f_1st', post
est store suicide_f_`x'_1st

** Add 1st stage statistics
est restore tmp_suicide_f_`x'
estadd scalar F = `e(widstat)' : suicide_f_`x'_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_f_`x'_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_`x'_1st

** Reduced form
reghdfe D.suicide_f_`x'_log commuter_ratio_m, cluster(`clsvars')
nlcom `paracomp_f_reduced', post
est store suicide_f_`x'_reduced

/* CV */
* FE
reghdfe D.suicide_f_`x'_log D.cv_residents, cluster(`clsvars')
nlcom `paracomp_f_cv', post
est store suicide_f_`x'_cv_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_f_`x'_log (D.cv_residents=commuter_ratio_m), gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_f_`x'_cv

nlcom `paracomp_f_cv', post
est store suicide_f_`x'_cv_2nd
    
** Save 1st stage result
est restore F_1st_D_cv_residents
nlcom `paracomp_f_cv_1st', post
est store suicide_f_`x'_cv_1st

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_cv
estadd scalar F = `e(widstat)' : suicide_f_`x'_cv_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_f_`x'_cv_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_`x'_cv_1st    
      
** Reduced form
reghdfe D.suicide_f_log commuter_ratio_m, cluster(`clsvars')
nlcom `paracomp_f_cv_reduced', post
est store suicide_f_`x'_cv_reduced      
                                                                                               
#delimit ;
noi: di "******* Female `x' *******";                                                                                               
noi: esttab suicide_f_`x'_FE suicide_f_`x'_1st suicide_f_`x'_2nd suicide_f_`x'_reduced suicide_f_`x'_cv_FE suicide_f_`x'_cv_1st suicide_f_`x'_cv_2nd suicide_f_`x'_cv_reduced, 
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) label rename(cv_residents mobility_index)
keep(mobility_index commuter_ratio_m) order(mobility_index commuter_ratio_m)                                                                                              
mgroups("Outflow Ratio" "CV", pattern(1 0 0 0 1 0 0 0)) mlabels("OLS" "1st Stage" "IV" "Reduced form" "OLS" "1st Stage" "IV" "Reduced form") varwidth(4) modelwidth(4)
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation"));                                                                                                                                                                                     
 
* Redefine label                                                                                 
label variable mobility_index "Outflow ratio";
label variable cv_residents "CV";                                                                                         
                                                                                               
};                                                                                                                                                                                     
};


. qui{
******* Female u20 *******

--------------------------------------------------------------------
     Out~o                              CV                           
      (1)     (2)     (3)     (4)     (5)     (6)     (7)     (8)   
      OLS    1st~e      IV    Red~m     OLS    1st~e      IV    Red~m   
--------------------------------------------------------------------
Mob~x -2.961**          -5.512**          -0.373***         -0.622**         
     (1.159)            (2.172)            (0.140)            (0.245)          
>  

Mal~o         -0.134***         0.741**          -1.191***         0.611   
             (0.005)            (0.292)            (0.040)            (0.479)  
>  
--------------------------------------------------------------------
F s~c         778.128                            879.439                   
Eff~c         778.128                            879.439                   
tau~%         23.109                            23.109                

In [21]:
%%stata
* Export stacked tables
qui{ 

* Define label temporary    
label variable mobility_index "Mobility index"
label variable cv_residents "Mobility index"
    
* female u20
esttab suicide_f_u20_FE suicide_f_u20_1st suicide_f_u20_2nd suicide_f_u20_reduced suicide_f_u20_cv_FE suicide_f_u20_cv_1st suicide_f_u20_cv_2nd suicide_f_u20_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_without_covariates_female.tex", ///
prehead("\begin{tabular}{l*{8}{c}} \toprule") ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(A)Under 20}}} \\\\[-1ex]") ///    
fragment ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index commuter_ratio_m) order(mobility_index commuter_ratio_m) ///                                                                                                                                                                                                                                                                                 
mgroups("Outflow Ratio" "CV", pattern(1 0 0 0 1 0 0 0) prefix(\multicolumn{@span}{c}{) suffix(}) span erepeat(\cmidrule(lr){@span})) ///
mlabels("OLS" "1st Stage" "IV" "Reduced form" "OLS" "1st Stage" "IV" "Reduced form") ///                            
nomtitles noobs label rename(cv_residents mobility_index) ///
booktabs alignment(D{.}{.}{-1}) ///                                                                                  
replace
                                                                                 
* female 20-40
esttab suicide_f_20_40_FE suicide_f_20_40_1st suicide_f_20_40_2nd suicide_f_20_40_reduced suicide_f_20_40_cv_FE suicide_f_20_40_cv_1st suicide_f_20_40_cv_2nd suicide_f_20_40_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_without_covariates_female.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(B)20-39}}} \\\\[-1ex]") ///
fragment ///   
append ///                                                                                    
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index commuter_ratio_m) order(mobility_index commuter_ratio_m) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers noobs label nolines rename(cv_residents mobility_index) ///   
booktabs alignment(D{.}{.}{-1}) ///                                                                                  
                                                                                 
* female 40-60
esttab suicide_f_40_60_FE suicide_f_40_60_1st suicide_f_40_60_2nd suicide_f_40_60_reduced suicide_f_40_60_cv_FE suicide_f_40_60_cv_1st suicide_f_40_60_cv_2nd suicide_f_40_60_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_without_covariates_female.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(C)40-59}}} \\\\[-1ex]") ///
fragment ///   
append ///                                                                                    
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index commuter_ratio_m) order(mobility_index commuter_ratio_m) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers noobs label nolines rename(cv_residents mobility_index) /// 
booktabs alignment(D{.}{.}{-1}) ///                                                                                 
                                                                                 
                                                                                     
* female o60                                                                                  
esttab suicide_f_o60_FE suicide_f_o60_1st suicide_f_o60_2nd suicide_f_o60_reduced suicide_f_o60_cv_FE suicide_f_o60_cv_1st suicide_f_o60_cv_2nd suicide_f_o60_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_without_covariates_female.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(D)Over 60}}} \\\\[-1ex]") ///
fragment ///
append ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///           
keep(mobility_index commuter_ratio_m) order(mobility_index commuter_ratio_m) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers label rename(cv_residents mobility_index) ///
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation")) ///                                                                                    
prefoot("\hline") ///                                                                                   
postfoot("\bottomrule \multicolumn{9}{l}{\footnotesize Standard errors in parentheses}\\ \multicolumn{8}{l}{\footnotesize \sym{*} \(p<0.10\), \sym{**} \(p<0.05\), \sym{***} \(p<0.01\)} \end{tabular}") ///
booktabs alignment(D{.}{.}{-1})

        
* Redefine label                                                                                 
label variable mobility_index "Outflow ratio"
label variable cv_residents "CV"
                                                                                 
}



. * Export stacked tables
. qui{ 

. 


# Male 

## Male: Total

In [22]:
%%stata

qui{
/* Outflow ratio */
* FE
reghdfe D.suicide_m_log D.mobility_index, cluster(`clsvars')
nlcom `paracomp_m_mobility', post
est store suicide_m_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_m_log (D.mobility_index=commuter_ratio_f  ), gmm2s cluster(`clsvars') first savefirst savefprefix(M_1st_)
est store tmp_suicide_m

** Save 2nd stage result
nlcom `paracomp_m_mobility', post
est store suicide_m_2nd

** Save 1st stage result
est restore M_1st_D_mobility_index
nlcom `paracomp_m_1st', post
est store suicide_m_1st

** Add 1st stage statistics
est restore tmp_suicide_m
estadd scalar F = `e(widstat)' : suicide_m_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_m_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_m_1st

* Reduced form
reghdfe D.suicide_m_log commuter_ratio_f, cluster(`clsvars')
nlcom `paracomp_m_reduced', post
est store suicide_m_reduced

/* CV */
* FE
reghdfe D.suicide_m_log D.cv_residents, cluster(`clsvars')
nlcom `paracomp_m_cv', post
est store suicide_m_cv_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_m_log (D.cv_residents=commuter_ratio_f  ), gmm2s cluster(`clsvars') first savefirst savefprefix(M_1st_)
est store tmp_suicide_m_cv

** Save 2nd stage result
nlcom `paracomp_m_cv', post
est store suicide_m_cv_2nd

** Save 1st stage result
est restore M_1st_D_cv_residents
nlcom `paracomp_m_cv_1st', post
est store suicide_m_cv_1st

** Add 1st stage statistics
est restore tmp_suicide_m_cv
estadd scalar F = `e(widstat)' : suicide_m_cv_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_m_cv_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_m_cv_1st

* Reduced form
reghdfe D.suicide_m_log commuter_ratio_f, cluster(`clsvars')
nlcom `paracomp_m_cv_reduced', post
est store suicide_m_cv_reduced                                                                                               
 
#delimit ;                                                                                               
                                                                                               
noi: esttab suicide_m_FE suicide_m_1st suicide_m_2nd suicide_m_reduced, 
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) label 
mgroups("OLS" "1st Stage" "IV" "Reduced form", pattern(1 1 1 1)) mlabels("Log(Male suicide)" "Outflow ratio" "Log(Male suicide)" "Log(Male suicide)", lhs("Dependent variables: ")) 
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation"))  varwidth(10) modelwidth(10);
 
noi: esttab suicide_m_cv_FE suicide_m_cv_1st suicide_m_cv_2nd suicide_m_cv_reduced, 
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) label 
mgroups("OLS" "1st Stage" "IV" "Reduced form", pattern(1 1 1 1)) mlabels("Log(Male suicide)" "CV" "Log(Male suicide)" "Log(Male suicide)", lhs("Dependent variables: ")) 
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation"))  varwidth(10) modelwidth(10);
                                                                                                                                                                                        
                                                                                              
                                                                                              
}; 


. 
. qui{

------------------------------------------------------------------
                  OLS     1st Stage            IV    Reduced ~m   
                  (1)           (2)           (3)           (4)   
Dependen~: Log(Male~)    Outflow ~o    Log(Male~)    Log(Male~)   
------------------------------------------------------------------
Outflow ~o     -0.483**                    -0.725                 
              (0.244)                     (0.545)                 

Female c~o                   -0.135***                    0.098   
                            (0.005)                     (0.074)   
------------------------------------------------------------------
F statis~c                  627.137                               
Effectiv~c                  627.137                               
tau=10\%                     23.109                               
Observat~n       1891          1891          1891          1891   
-------------------------------------------------

In [23]:
%%stata
* Export stacked tables

qui{                                                                     
esttab suicide_m_FE suicide_m_1st suicide_m_2nd suicide_m_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_male.tex", ///
prehead("\begin{tabular}{l*{4}{c}} \toprule") ///
posthead("\hline \\ \multicolumn{5}{l}{\textbf{\underline{(A)Outflow ratio}}} \\\\[-1ex]") ///
fragment ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
mgroups("OLS" "1st Stage" "IV" "Reduced form", ///
pattern(1 1 1 1) ///
prefix(\multicolumn{@span}{c}{) suffix(}) span erepeat(\cmidrule(lr){@span})) ///
mlabels("Log(Male suicide)" "Mobility index" "Log(Male suicide)" "Log(Male suicide)", lhs("Dependent variables: ")) ///                                  
nomtitles noobs label ///
booktabs alignment(D{.}{.}{-1}) ///                                
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observations")) ///                            
replace

* Bottom panel
esttab suicide_m_cv_FE suicide_m_cv_1st suicide_m_cv_2nd suicide_m_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_male.tex", ///
posthead("\hline \\ \multicolumn{5}{l}{\textbf{{\underline{(B)CV}}}} \\\\[-1ex]") ///
fragment ///
append ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///                                   
nomtitles nonumbers ///
booktabs alignment(D{.}{.}{-1}) ///                                
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observations")) ///
postfoot("\bottomrule \multicolumn{5}{l}{\footnotesize Standard errors in parentheses}\\ \multicolumn{5}{l}{\footnotesize \sym{*} \(p<0.10\), \sym{**} \(p<0.05\), \sym{***} \(p<0.01\)} \end{tabular}") ///
label                                                                                              
}


. * Export stacked tables
. 
. qui{                                                                     

. 


## Male: All age group without covariates

In [24]:
%%stata
qui{
foreach x in u20 20_40 40_60 o60{
sum suicide_m_`x'
local suicide_m_m=r(mean)
    
* Define label temporary    
label variable mobility_index "Mobility index"
label variable cv_residents "Mobility index"

#delimit ;
local paracomp_m_mobility="(mobility_index: _b[D.mobility_index]*(1+`suicide_m_m')/(`suicide_m_m'))";
local paracomp_m_cv="(cv_residents: _b[D.cv_residents]*(1+`suicide_m_m')/(`suicide_m_m'))";
local paracomp_m_cv_1st="(commuter_ratio_f: _b[commuter_ratio_f])";
local paracomp_m_1st="(commuter_ratio_f: _b[commuter_ratio_f])";
local paracomp_m_cv_reduced="(commuter_ratio_f: _b[commuter_ratio_f]*(1+`suicide_m_m')/(`suicide_m_m')) ";
local paracomp_m_reduced="(commuter_ratio_f: _b[commuter_ratio_f]*(1+`suicide_m_m')/(`suicide_m_m'))";

#delimit cr
/* Outflow ratio */
* FE
reghdfe D.suicide_m_`x'_log D.mobility_index, cluster(`clsvars')
nlcom `paracomp_m_mobility', post
est store suicide_m_`x'_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_m_`x'_log (D.mobility_index=commuter_ratio_f), gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_m_`x'
    
nlcom `paracomp_m_mobility', post
est store suicide_m_`x'_2nd
    
** Save 1st stage result
est restore F_1st_D_mobility_index
nlcom `paracomp_m_1st', post
est store suicide_m_`x'_1st

** Add 1st stage statistics
est restore tmp_suicide_m_`x'
estadd scalar F = `e(widstat)' : suicide_m_`x'_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_m_`x'_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_m_`x'_1st

** Reduced form
reghdfe D.suicide_m_`x'_log commuter_ratio_f, cluster(`clsvars')
nlcom `paracomp_m_reduced', post
est store suicide_m_`x'_reduced

/* CV */
* FE
reghdfe D.suicide_m_`x'_log D.cv_residents, cluster(`clsvars')
nlcom `paracomp_m_cv', post
est store suicide_m_`x'_cv_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_m_`x'_log (D.cv_residents=commuter_ratio_f), gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_m_`x'_cv

nlcom `paracomp_m_cv', post
est store suicide_m_`x'_cv_2nd
    
** Save 1st stage result
est restore F_1st_D_cv_residents
nlcom `paracomp_m_cv_1st', post
est store suicide_m_`x'_cv_1st

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_cv
estadd scalar F = `e(widstat)' : suicide_m_`x'_cv_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_m_`x'_cv_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_m_`x'_cv_1st    
      

** Reduced form
reghdfe D.suicide_m_log commuter_ratio_f, cluster(`clsvars')
nlcom `paracomp_m_cv_reduced', post
est store suicide_m_`x'_cv_reduced      
                                                                                               
#delimit ;
noi: di "******* Male `x' *******";                                                                                               
noi: esttab suicide_m_`x'_FE suicide_m_`x'_1st suicide_m_`x'_2nd suicide_m_`x'_reduced suicide_m_`x'_cv_FE suicide_m_`x'_cv_1st suicide_m_`x'_cv_2nd suicide_m_`x'_cv_reduced, 
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) label rename(cv_residents mobility_index)
keep(mobility_index commuter_ratio_f) order(mobility_index commuter_ratio_f)                                                                                              
mgroups("Outflow Ratio" "CV", pattern(1 0 0 0 1 0 0 0)) mlabels("OLS" "1st Stage" "IV" "Reduced form" "OLS" "1st Stage" "IV" "Reduced form") varwidth(4) modelwidth(4)
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation"));   
                                                                                                                 
* Redefine label                                                                                 
label variable mobility_index "Outflow ratio";
label variable cv_residents "CV";                                                                                         
                                                                                               
};                                                                                                                                                                                     
};


. qui{
******* Male u20 *******

--------------------------------------------------------------------
     Out~o                              CV                           
      (1)     (2)     (3)     (4)     (5)     (6)     (7)     (8)   
      OLS    1st~e      IV    Red~m     OLS    1st~e      IV    Red~m   
--------------------------------------------------------------------
Mob~x -1.053            -1.530            -0.115            -0.171           
     (0.876)            (1.684)            (0.106)            (0.188)          
>  

Fem~o         -0.135***         0.207            -1.207***         0.442   
             (0.005)            (0.228)            (0.045)            (0.333)  
>  
--------------------------------------------------------------------
F s~c         627.137                            719.525                   
Eff~c         627.137                            719.525                   
tau~%         23.109                            23.109                  

In [25]:
%%stata
* Export stacked tables
qui{ 

* Define label temporary    
label variable mobility_index "Mobility index"
label variable cv_residents "Mobility index"
    
* male u20
esttab suicide_m_u20_FE suicide_m_u20_1st suicide_m_u20_2nd suicide_m_u20_reduced suicide_m_u20_cv_FE suicide_m_u20_cv_1st suicide_m_u20_cv_2nd suicide_m_u20_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_without_covariates_male.tex", ///
prehead("\begin{tabular}{l*{8}{c}} \toprule") ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(A)Under 20}}} \\\\[-1ex]") ///    
fragment ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index commuter_ratio_f) order(mobility_index commuter_ratio_f) ///                                                                                                                                                                                                                                                                                 
mgroups("Outflow Ratio" "CV", pattern(1 0 0 0 1 0 0 0) prefix(\multicolumn{@span}{c}{) suffix(}) span erepeat(\cmidrule(lr){@span})) ///
mlabels("OLS" "1st Stage" "IV" "Reduced form" "OLS" "1st Stage" "IV" "Reduced form") ///                            
nomtitles noobs label rename(cv_residents mobility_index) ///
booktabs alignment(D{.}{.}{-1}) ///                                                                                
replace
                                                                                 
* male 20-40
esttab suicide_m_20_40_FE suicide_m_20_40_1st suicide_m_20_40_2nd suicide_m_20_40_reduced suicide_m_20_40_cv_FE suicide_m_20_40_cv_1st suicide_m_20_40_cv_2nd suicide_m_20_40_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_without_covariates_male.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(B)20-39}}} \\\\[-1ex]") ///
fragment ///   
append ///                                                                                    
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index commuter_ratio_f) order(mobility_index commuter_ratio_f) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers noobs label nolines rename(cv_residents mobility_index) ///  
booktabs alignment(D{.}{.}{-1}) ///                                                                                        
                                                                                 
* male 40-60
esttab suicide_m_40_60_FE suicide_m_40_60_1st suicide_m_40_60_2nd suicide_m_40_60_reduced suicide_m_40_60_cv_FE suicide_m_40_60_cv_1st suicide_m_40_60_cv_2nd suicide_m_40_60_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_without_covariates_male.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(C)40-59}}} \\\\[-1ex]") ///
fragment ///   
append ///                                                                                    
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index commuter_ratio_f) order(mobility_index commuter_ratio_f) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers noobs label nolines rename(cv_residents mobility_index) ///  
booktabs alignment(D{.}{.}{-1}) ///                                                                                        
                                                                                     
* male o60                                                                                  
esttab suicide_m_o60_FE suicide_m_o60_1st suicide_m_o60_2nd suicide_m_o60_reduced suicide_m_o60_cv_FE suicide_m_o60_cv_1st suicide_m_o60_cv_2nd suicide_m_o60_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_without_covariates_male.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(D)Over 60}}} \\\\[-1ex]") ///
fragment ///
append ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///           
keep(mobility_index commuter_ratio_f) order(mobility_index commuter_ratio_f) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers label rename(cv_residents mobility_index) ///  
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation")) ///                                                                                     
prefoot("\hline") ///                                                                                   
postfoot("\bottomrule \multicolumn{9}{l}{\footnotesize Standard errors in parentheses}\\ \multicolumn{8}{l}{\footnotesize \sym{*} \(p<0.10\), \sym{**} \(p<0.05\), \sym{***} \(p<0.01\)} \end{tabular}") ///
booktabs alignment(D{.}{.}{-1}) 

        
* Redefine label                                                                                 
label variable mobility_index "Outflow ratio"
label variable cv_residents "CV"
                                                                                 
}



. * Export stacked tables
. qui{ 

. 


## Male: All age group with covariates

In [26]:
%%stata
qui{
foreach x in u20 20_40 40_60 o60{
sum suicide_m_`x'
local suicide_m_m=r(mean)
    
* Define label temporary    
label variable mobility_index "Mobility index"
label variable cv_residents "Mobility index"

#delimit ;
local paracomp_m_mobility_full="(mobility_index: _b[D.mobility_index]*(1+`suicide_m_m')/(`suicide_m_m')) 
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))*((1+`suicide_m_m')/(`suicide_m_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))*((1+`suicide_m_m')/(`suicide_m_m'))) ";
local paracomp_m_cv_full="(cv_residents: _b[D.cv_residents]*(1+`suicide_m_m')/(`suicide_m_m')) 
    (cv_nonresidents: _b[D.cv_nonresidents]*(1+`suicide_m_m')/(`suicide_m_m'))
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))*((1+`suicide_m_m')/(`suicide_m_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))*((1+`suicide_m_m')/(`suicide_m_m'))) ";
local paracomp_m_cv_1st_full="(commuter_ratio_f: _b[commuter_ratio_f])
    (cv_nonresidents: _b[D.cv_nonresidents]*(1+`suicide_m_m')/(`suicide_m_m'))
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m')))
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))) ";
local paracomp_m_1st_full="(commuter_ratio_f: _b[commuter_ratio_f])
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m')))
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))) ";
local paracomp_m_cv_reduced_full="(commuter_ratio_f: _b[commuter_ratio_f]*(1+`suicide_m_m')/(`suicide_m_m'))
    (cv_nonresidents: _b[D.cv_nonresidents]*(1+`suicide_m_m')/(`suicide_m_m'))
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))*((1+`suicide_m_m')/(`suicide_m_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))*((1+`suicide_m_m')/(`suicide_m_m'))) ";
local paracomp_m_reduced_full="(commuter_ratio_f: _b[commuter_ratio_f]*(1+`suicide_m_m')/(`suicide_m_m'))     
    (`log_'active_opening_ratio: _b[D.`log_'active_opening_ratio]*((`log_active_opening_ratio_m')/(1+`log_active_opening_ratio_m'))*((1+`suicide_m_m')/(`suicide_m_m'))) 
    (`log_'patient: _b[D.`log_'patient]*((`log_patient_m')/(1+`log_patient_m'))*((1+`suicide_m_m')/(`suicide_m_m'))) ";

#delimit cr
/* Outflow ratio */
* FE
reghdfe D.suicide_m_`x'_log D.mobility_index D.`log_'active_opening_ratio D.`log_'patient, cluster(`clsvars')
nlcom `paracomp_m_mobility_full', post
est store suicide_m_`x'_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_m_`x'_log (D.mobility_index=commuter_ratio_f  ) D.`log_'active_opening_ratio D.`log_'patient, gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_m_`x'
    
nlcom `paracomp_m_mobility_full', post
est store suicide_m_`x'_2nd
    
** Save 1st stage result
est restore F_1st_D_mobility_index
nlcom `paracomp_m_1st_full', post
est store suicide_m_`x'_1st

** Add 1st stage statistics
est restore tmp_suicide_m_`x'
estadd scalar F = `e(widstat)' : suicide_m_`x'_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_m_`x'_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_m_`x'_1st

** Reduced form
reghdfe D.suicide_m_`x'_log commuter_ratio_f D.`log_'active_opening_ratio D.`log_'patient, cluster(`clsvars')
nlcom `paracomp_m_reduced_full', post
est store suicide_m_`x'_reduced

/* CV */
* FE
reghdfe D.suicide_m_`x'_log D.cv_residents D.cv_nonresidents D.`log_'active_opening_ratio D.`log_'patient, cluster(`clsvars')
nlcom `paracomp_m_cv_full', post
est store suicide_m_`x'_cv_FE

* IV (1st & 2nd Stage)
ivreg2 D.suicide_m_`x'_log (D.cv_residents=commuter_ratio_f  ) D.cv_nonresidents D.`log_'active_opening_ratio D.`log_'patient, gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_m_`x'_cv

nlcom `paracomp_m_cv_full', post
est store suicide_m_`x'_cv_2nd
    
** Save 1st stage result
est restore F_1st_D_cv_residents
nlcom `paracomp_m_cv_1st_full', post
est store suicide_m_`x'_cv_1st

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_cv
estadd scalar F = `e(widstat)' : suicide_m_`x'_cv_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_m_`x'_cv_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_m_`x'_cv_1st    
      

** Reduced form
reghdfe D.suicide_m_log commuter_ratio_f D.cv_nonresidents D.`log_'active_opening_ratio D.`log_'patient, cluster(`clsvars')
nlcom `paracomp_m_cv_reduced_full', post
est store suicide_m_`x'_cv_reduced      
                                                                                               
#delimit ;
noi: di "******* Male `x' *******";                                                                                               
noi: esttab suicide_m_`x'_FE suicide_m_`x'_1st suicide_m_`x'_2nd suicide_m_`x'_reduced suicide_m_`x'_cv_FE suicide_m_`x'_cv_1st suicide_m_`x'_cv_2nd suicide_m_`x'_cv_reduced, 
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) label rename(cv_residents mobility_index)
keep(mobility_index cv_nonresidents commuter_ratio_f `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_f)                                                                                              
mgroups("Outflow Ratio" "CV", pattern(1 0 0 0 1 0 0 0)) mlabels("OLS" "1st Stage" "IV" "Reduced form" "OLS" "1st Stage" "IV" "Reduced form") varwidth(4) modelwidth(4)
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation"));   
                                                                                                                 
* Redefine label                                                                                 
label variable mobility_index "Outflow ratio";
label variable cv_residents "CV";                                                                                         
                                                                                               
};                                                                                                                                                                                     
};


. qui{
******* Male u20 *******

--------------------------------------------------------------------
     Out~o                              CV                           
      (1)     (2)     (3)     (4)     (5)     (6)     (7)     (8)   
      OLS    1st~e      IV    Red~m     OLS    1st~e      IV    Red~m   
--------------------------------------------------------------------
Mob~x -0.706            -0.885            -0.063            -0.068           
     (0.813)            (2.003)            (0.100)            (0.226)          
>  

CV ~s                                 -0.061    0.160    -0.060    0.012   
                                     (0.045)    (0.107)    (0.045)    (0.128)  
>  

Log~) 0.064    0.006*** 0.067    0.061    0.065    0.053*** 0.065    -0.173*  
     (0.061)    (0.002)    (0.067)    (0.063)    (0.062)    (0.013)    (0.067) 
>    (0.095)   

Log~) 0.043**  -0.004*** 0.042*   0.045**  0.042**  -0.035*** 0.041*   0.025   
     (0.021)    (0.001)    (0.023)  

In [27]:
%%stata
* Export stacked tables
qui{ 

* Define label temporary    
label variable mobility_index "Mobility index"
label variable cv_residents "Mobility index"
    
* male u20
esttab suicide_m_u20_FE suicide_m_u20_1st suicide_m_u20_2nd suicide_m_u20_reduced suicide_m_u20_cv_FE suicide_m_u20_cv_1st suicide_m_u20_cv_2nd suicide_m_u20_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_male.tex", ///
prehead("\begin{tabular}{l*{8}{c}} \toprule") ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(A)Under 20}}} \\\\[-1ex]") ///    
fragment ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index cv_nonresidents commuter_ratio_f `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_f) ///                                                                                                                                                                                                                                                                                 
mgroups("Outflow Ratio" "CV", pattern(1 0 0 0 1 0 0 0) prefix(\multicolumn{@span}{c}{) suffix(}) span erepeat(\cmidrule(lr){@span})) ///
mlabels("OLS" "1st Stage" "IV" "Reduced form" "OLS" "1st Stage" "IV" "Reduced form") ///                            
nomtitles noobs label rename(cv_residents mobility_index) ///
booktabs alignment(D{.}{.}{-1}) ///                                                                                
replace
                                                                                 
* male 20-40
esttab suicide_m_20_40_FE suicide_m_20_40_1st suicide_m_20_40_2nd suicide_m_20_40_reduced suicide_m_20_40_cv_FE suicide_m_20_40_cv_1st suicide_m_20_40_cv_2nd suicide_m_20_40_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_male.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(B)20-39}}} \\\\[-1ex]") ///
fragment ///   
append ///                                                                                    
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index cv_nonresidents commuter_ratio_f `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_f) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers noobs label nolines rename(cv_residents mobility_index) ///  
booktabs alignment(D{.}{.}{-1}) ///                                                                                        
                                                                                 
* male 40-60
esttab suicide_m_40_60_FE suicide_m_40_60_1st suicide_m_40_60_2nd suicide_m_40_60_reduced suicide_m_40_60_cv_FE suicide_m_40_60_cv_1st suicide_m_40_60_cv_2nd suicide_m_40_60_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_male.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(C)40-59}}} \\\\[-1ex]") ///
fragment ///   
append ///                                                                                    
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///
keep(mobility_index cv_nonresidents commuter_ratio_f `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_f) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers noobs label nolines rename(cv_residents mobility_index) ///  
booktabs alignment(D{.}{.}{-1}) ///                                                                                        
                                                                                     
* male o60                                                                                  
esttab suicide_m_o60_FE suicide_m_o60_1st suicide_m_o60_2nd suicide_m_o60_reduced suicide_m_o60_cv_FE suicide_m_o60_cv_1st suicide_m_o60_cv_2nd suicide_m_o60_cv_reduced using "../figuretable/appendix/baseline_cv_outflowratio_crossgender_fullresult_male.tex", ///
posthead("\hline \\ \multicolumn{9}{l}{\textbf{\underline{(D)Over 60}}} \\\\[-1ex]") ///
fragment ///
append ///
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01) ///           
keep(mobility_index cv_nonresidents commuter_ratio_f `log_'active_opening_ratio `log_'patient) order(mobility_index cv_nonresidents `log_'active_opening_ratio `log_'patient commuter_ratio_f) ///                                                                                                                                                                                                                                                                                                                                                                  
nomtitles nonumbers label rename(cv_residents mobility_index) ///  
stats(F F_eff c_TSLS_10 N, layout(@ @ @ @) fmt(3 3 3 %5.0f) labels("F statistic" "Effective F statistic" "tau=10\%" "Observation")) ///                                                                                     
prefoot("\hline") ///                                                                                   
postfoot("\bottomrule \multicolumn{9}{l}{\footnotesize Standard errors in parentheses}\\ \multicolumn{8}{l}{\footnotesize \sym{*} \(p<0.10\), \sym{**} \(p<0.05\), \sym{***} \(p<0.01\)} \end{tabular}") ///
booktabs alignment(D{.}{.}{-1}) 

        
* Redefine label                                                                                 
label variable mobility_index "Outflow ratio"
label variable cv_residents "CV"
                                                                                 
}



. * Export stacked tables
. qui{ 

. 


## Counterfactual

In [28]:
%%stata
qui{
local IV_1="commuter_ratio_m_iv "
local IV_2="commuter_ratio_iv "
local IV_3="commuter_ratio_m_iv commuter_ratio_f_iv "

matrix define CF = (0, 0, 0 \ 0, 0, 0)
    
forvalues i = 1(1)3{

* Day to night ratio
ivreghdfe suicide_f_u20_log (mobility_index=`IV_`i''), gmm2s absorb(`absvars', savefe) resid cluster(`clsvars')  
predict pred_f_iv`i', xbd
gen pred_num_f_iv`i' = exp(pred_f_iv`i') - 1             
bysort citycode: gen cf_f_iv`i' =  exp(_b[mobility_index] * mobility_index + __hdfe1__[_n+1] + __hdfe2__[_n+1] ) - 1   
bysort citycode: gen pred_cf_diff_f_iv`i' = cf_f_iv`i' - pred_num_f_iv`i'[_n+1]                                                                                                                                  
sum pred_cf_diff_f_iv`i'
scalar total_pred_cf_diff_f_iv`i' = r(sum)
sum pred_num_f_iv`i' if covid == 1   
scalar total_pred_num_f_iv`i' = r(sum)
sum cf_f_iv`i'
scalar total_pred_cf_f_iv`i' = r(sum)
scalar percent_change_cf`i' = (total_pred_cf_f_iv`i' - total_pred_num_f_iv`i')/total_pred_cf_f_iv`i'
matrix CF[1,`i']=percent_change_cf`i'*100
                                                                                                                                                 
* CV
ivreghdfe suicide_f_u20_log (cv_residents=`IV_`i''), gmm2s absorb(`absvars', savefe) resid cluster(`clsvars')  
predict pred_f_iv_cv`i', xbd
gen pred_num_f_iv_cv`i' = exp(pred_f_iv_cv`i') - 1             
bysort citycode: gen cf_f_iv_cv`i' =  exp(_b[cv_residents] * cv_residents + __hdfe1__[_n+1] + __hdfe2__[_n+1] ) - 1   
bysort citycode: gen pred_cf_diff_f_iv_cv`i' = cf_f_iv_cv`i' - pred_num_f_iv_cv`i'[_n+1]                                                                                                                                  
sum pred_cf_diff_f_iv_cv`i'
scalar total_pred_cf_diff_f_iv_cv`i' = r(sum)
sum pred_num_f_iv_cv`i' if covid == 1   
scalar total_pred_num_f_iv_cv`i' = r(sum)
sum cf_f_iv_cv`i'
scalar total_pred_cf_f_iv_cv`i' = r(sum)
scalar percent_change_cf_cv`i' = (total_pred_cf_f_iv_cv`i' - total_pred_num_f_iv_cv`i')/total_pred_cf_f_iv_cv`i'
matrix CF[2,`i']=percent_change_cf_cv`i'*100
}

matrix rownames CF = "Outflow ratio" "CV"
matrix colnames CF = "IV: Cross gender" "IV: Combined" "IV: Both"
matrix list CF

}

# delimit ;
esttab matrix(CF, fmt(%4.1f)), nomtitles;

esttab matrix(CF, fmt(%4.1f)) using "../figuretable/appendix/counterfactual_female_u20.tex", 
nomtitles booktabs replace;


. qui{

. 
. # delimit ;
delimiter now ;
. esttab matrix(CF, fmt(%4.1f)), nomtitles;

---------------------------------------------------
             IV:Cross g~r  IV:Combined      IV:Both
---------------------------------------------------
Outflow ra~o        -42.7        -38.5        -48.2
CV                  -32.3        -29.3        -35.5
---------------------------------------------------

. esttab matrix(CF, fmt(%4.1f)) using "../figuretable/appendix/counterfactual_f
> emale_u20.tex", 
> nomtitles booktabs replace;
(file ../figuretable/appendix/counterfactual_female_u20.tex not found)
(output written to ../figuretable/appendix/counterfactual_female_u20.tex)

. 
